In [1]:
from google.colab import drive
drive.mount("/content/drive/")


root = '/content/drive/My Drive/NLP'

Mounted at /content/drive/


In [31]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed May 31 20:29:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    29W /  70W |   1919MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [78]:
import os
sts_path = os.path.join(root, 'stsb/')
print(sts_path)

/content/drive/My Drive/NLP/stsb/


In [79]:
!pip3 install transformers==4.25.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [80]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import torch
import seaborn as sns
import transformers
import pandas as pd
import seaborn as sns
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

import transformers
from transformers import AutoModelForMaskedLM
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup,get_cosine_schedule_with_warmup
from transformers import get_cosine_with_hard_restarts_schedule_with_warmup
from transformers import RobertaTokenizer, RobertaModel, AdamW, get_linear_schedule_with_warmup, RobertaConfig

import logging
logging.basicConfig(level=logging.ERROR)
from ipywidgets import IntProgress
import time

In [113]:
# Set random seed and set device to GPU.
torch.manual_seed(17)

if torch.cuda.is_available():
    device = torch.device('cuda:0')
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    device = torch.device('cpu')

print(device)
import gc
torch.cuda.empty_cache()
gc.collect()

cuda:0


67

In [139]:
NUM_WORKERS = 0
TRAIN_BATCH_SIZE = 32
EPOCHS = 12
SEED = 2023
LR = 1e-5

device = torch.device('cuda')

transformer_model = 'roberta-base'
TOKENIZER = RobertaTokenizer.from_pretrained(transformer_model)
CONFIG = RobertaConfig.from_pretrained(transformer_model)

mask_tok = 50264

In [128]:
class AverageMeter(object):
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [129]:
def save_metrics(path, train_loss_list, valid_loss_list, global_steps_list):   
    state_dict = {'train_loss_list': train_loss_list,
                  'valid_loss_list': valid_loss_list,
                  'global_steps_list': global_steps_list}
    torch.save(state_dict, path)

def load_metrics(path):    
    state_dict = torch.load(path, map_location=device)
    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']

In [130]:
def save_model(model, path):
    torch.save(model.state_dict(), path)

In [131]:
# from https://www.kaggle.com/code/tanulsingh077/masked-token-pretraining-xlm-roberta/notebook
class Data(Dataset):
    def __init__(self, csv):
        self.csv = csv.reset_index()

    def __len__(self):
        return self.csv.shape[0]

    def __getitem__(self, index):
        row = self.csv.iloc[index]
        text = row.text
        
        text = TOKENIZER(text,
                         return_attention_mask=False,
                         return_token_type_ids=False,
                         padding='max_length',
                         truncation=True,
                         max_length=64)
        
        input_ids = text['input_ids']
        
        input_ids,labels = self.prepare_mlm_input_and_labels(np.array(input_ids))

        input_ids = torch.tensor(input_ids,dtype=torch.long)
        labels = torch.tensor(labels,dtype=torch.long)
    
        return input_ids,labels
    
    def prepare_mlm_input_and_labels(self,X):
        # 15% BERT masking
        inp_mask = np.random.rand(*X.shape)<0.15 
        # do not mask special tokens
        inp_mask[X<=2] = False
        # set targets to -1 by default, it means ignore
        labels = -100 * np.ones(X.shape, dtype=int)
        # set labels for masked tokens
        labels[inp_mask] = X[inp_mask]
        
        # prepare input
        X_mlm = np.copy(X)
        # set input to [MASK] which is the last token for the 90% of tokens
        # this means leaving 10% unchanged
        inp_mask_2mask = inp_mask  & (np.random.rand(*X.shape)<0.90)
        X_mlm[inp_mask_2mask] = mask_tok

        # set 10% to a random token
        inp_mask_2random = inp_mask_2mask  & (np.random.rand(*X.shape) < 1/9)
        X_mlm[inp_mask_2random] = np.random.randint(3, CONFIG.vocab_size, inp_mask_2random.sum())

        return X_mlm, labels

In [132]:
from decimal import Decimal
# Load CSV file with dataset. Perform basic transformations.
df_train = pd.read_csv(sts_path+'/stsb-en-train.csv', delimiter=',', header= None, names = ['s1','s2','target'])
df_valid = pd.read_csv(sts_path+'/stsb-en-dev.csv', delimiter=',', header= None, names = ['s1','s2','target'])
df_test = pd.read_csv(sts_path+'/stsb-en-test.csv', delimiter=',', header= None, names = ['s1','s2','target'])

def process_sts(df_data):
  df_data['text'] = df_data['s1'] + df_data['s2']
  df_data = df_data[['text', 'target']].copy()
  df_data['target'] = df_data['target'].map(lambda x: int(Decimal(x).to_integral_value()))
  return df_data

df_train = process_sts(df_train)
df_valid = process_sts(df_valid)
df_test = process_sts(df_test)

In [133]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)

df_train = pd.concat([df_train, df_test, df_valid])
df_train

,text,target
0,A plane is taking off.An air plane is taking off.,5
1,A man is playing a large flute.A man is playin...,4
2,A man is spreading shreded cheese on a pizza.A...,4
3,Three men are playing chess.Two men are playin...,3
4,A man is playing the cello.A man seated is pla...,4
...,...,...
1495,Scientists prove there is water on MarsHas Nas...,2
1496,Pranab stresses need to strive for peace by na...,0
1497,Volkswagen skids into red in wake of pollution...,2
1498,Obama is right: Africa deserves better leaders...,0


In [134]:
class_value_counts = pd.DataFrame(df_train['target']).value_counts(sort=False)
    # odwrotne wagowanie klas
weights = ((1/class_value_counts.values) / np.sum(1/class_value_counts.values ))
print(weights)
class_weights = torch.FloatTensor(weights).cuda()

[0.20891525 0.1764098  0.14971781 0.11152136 0.10521493 0.24822086]


In [135]:
def masked_categorical_crossentropy(output,target):
    y_true_masked = target[target!= -100]
    y_pred_masked = output[target!= -100]
    loss =  torch.nn.CrossEntropyLoss(weight=class_weights)(y_pred_masked,y_true_masked)
    return loss

In [136]:
def train_fn(dataloader,model,optimizer,device,scheduler,epoch):
    model.train()
    loss_score = AverageMeter()
    
    tk0 = tqdm(enumerate(dataloader), total=len(dataloader))
    for bi,d in tk0:
        
        batch_size = d[0].shape[0]

        input_ids = d[0]
        targets = d[1]

        input_ids = input_ids.to(device,dtype=torch.long)
        targets = targets.to(device)

        optimizer.zero_grad()

        output = model(input_ids=input_ids,labels=targets)
        
        loss = output.loss       
        
        loss.backward()
        optimizer.step()
        
        loss_score.update(loss.detach().item(), batch_size)
        tk0.set_postfix(Train_Loss=loss_score.avg,Epoch=epoch,LR=optimizer.param_groups[0]['lr'])
        
        if scheduler is not None:
                scheduler.step()
        
    return loss_score

In [137]:
def run():
    train_dataset = Data(
        csv=df_train
    )
        
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        pin_memory=True,
        drop_last=True,
        num_workers=NUM_WORKERS
    )
    
    
    model = RobertaForMaskedLM.from_pretrained(transformer_model)
    model.to(device)

        
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.0001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
            ]  
    
    optimizer = AdamW(optimizer_parameters, lr=LR)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=len(train_loader)*5, 
        num_training_steps=len(train_loader)*EPOCHS
    )
        
    # THE ENGINE LOOP
    for epoch in range(EPOCHS):
        train_loss = train_fn(train_loader, model,optimizer, device,scheduler=scheduler,epoch=epoch)


    model.save_pretrained(root+'/roberta_unsp_base')
    save_model(model, root + '/roberta_unsp_classifier')

In [140]:
run()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 269/269 [02:15<00:00,  1.98it/s, Epoch=11, LR=5.31e-9, Train_Loss=1.2]
